<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/126_%E3%82%B9%E3%83%AC%E3%83%83%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

スレッド
========

マルチコアとマルチスレッド
--------------------------

### 並行処理と並列処理

現代の CPU の内部には、独立して処理作業を行う回路のブロックが複数組み込まれており、それぞれが単体の処理装置のように振る舞う。これらを **CPU コア**と呼ぶ。CPU が複数の CPU コアを持つことを**マルチコア**（multi-core）と表現する。

CPU コアに処理させる対象が複数あって、処理を「同時に」行わせたい場合、CPU コアの処理形態には次の 2 種類がある。

  * **並行処理**（concurrent processing）:  
*1 個の CPU コア* が複数の処理を切り替えながら進めること。「複数の処理を切り替え」るとは、開始した処理を中断し、別の処理を再開することをいう。
  * **並列処理**（parallel processing）:  
*複数の CPU コア* が割り振られた処理を同時に進めること。

並行処理では、どの時間でも 1 個の CPU コアが 1 つの処理作業を行っているにすぎないが、処理の切り替えを素早く行うことによって「同時に」行っているように見せかける。これは、ワンオペレーションの飲食店で全ての作業が切り盛りされる様子に似ている（画像は[いらすとや](https://www.irasutoya.com/2018/06/blog-post_850.html)より）。

<img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEh5Dn4bGYN7AWxje4ecO8Ezh6Qqdn82HjlXE2iJtVj8Sl51a6DKoUTr5YrLloo8wRlf13YuzBPRAJ_mKW2TtYhOSIrAOiNg1WTBEpbMdf5enBQuuWnu6zeoxd2qhptHd-YYtUPwB01dKIA0/s800/job_one_operation_man.png" width="400">

並列処理では、複数の CPU コアを使って真の意味で「同時に」処理が進められるので、並行処理より確実に処理時間が短くなる。しかしながら、並列処理を行うには同時に行う処理の数に対して CPU コアの数が足りていることが必要であり、この条件がいつも満たされるとは限らない。

### プロセス

**プロセス**（process）は、プログラムを実行する実体（インスタンス）である。ハードディスク上に存在するプログラムがメモリ上に展開され、CPU で実行されている状態を指す。OS は、各プロセスを識別するために**プロセス ID**（process ID; PID）と呼ばれる一意な識別子を割り当てる。Linux では `ps` コマンド、Windows では `tasklist` コマンドで現在実行中のプロセスが一覧表示される。プロセスが終了すると、OS は割り当てたメモリを解放する。

In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      7 ?        00:00:00 node
     11 ?        00:00:00 oom_monitor.sh
     13 ?        00:00:00 run.sh
     14 ?        00:00:00 kernel_manager_
     37 ?        00:00:00 tail
     63 ?        00:00:06 python3 <defunct>
     64 ?        00:00:00 colab-fileshim.
     82 ?        00:00:03 jupyter-noteboo
     83 ?        00:00:00 dap_multiplexer
    599 ?        00:00:03 python3
    626 ?        00:00:00 python3
    654 ?        00:00:00 language_servic
    662 ?        00:00:18 node
    794 ?        00:00:00 sleep
    795 ?        00:00:00 ps


実行中のプロセスに関して、以下の Python 関数が利用できる。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `os.getpid()` | 実行中のプロセス ID を返す | `int` |
| `os.getppid()` | 親プロセス（実行中のプロセスを生成したプロセス）のプロセス ID を返す | `int` |

In [ ]:
import os
os.getpid(), os.getppid()

(599, 82)

Unix 系 OS では、実行中のプロセスに関する情報を取得・設定する以下の Python 関数が利用できる。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `os.getuid()` | 実行中のプロセスの実ユーザー ID を返す | `int` |
| `os.geteuid()` | 実行中のプロセスの実効ユーザー ID を返す | `int` |
| `os.getgid()` | 実行中のプロセスの実グループ ID を返す | `int` |
| `os.getegid()` | 実行中のプロセスの実効グループ ID を返す | `int` |
| `os.getgroups()` | 実行中のプロセスに関連付けられた従属グループ ID のリストを返す | `list` |
| `os.getpriority(which, who)` | プログラムのスケジューリング優先度を取得する。`which` は `PRIO_PROCESS`、`PRIO_PGRP`、または<br /><br /> `PRIO_USER` のいずれかとする。`who` は `which` に応じて解釈される（`PRIO_PROCESS` であればプロ<br /><br />セス ID、`PRIO_PGRP` であればプロセスグループ ID、そして `PRIO_USER` であればユーザー ID)。`who`<br /><br /> が `0` の場合、実行中のプロセス、そのグループと実ユーザー ID を意味する | `list` |
| `os.setuid(uid, /)` | 実行中のプロセスのユーザー id を設定する | `int` |
| `os.seteuid(euid, /)` | 実行中のプロセスに実効ユーザー ID をセットする | `None` |
| `os.setgid(gid, /)` | 実行中のプロセスにグループ ID をセットする | `None` |
| `os.setegid(egid, /)` | 実行中のプロセスに実効グループ ID をセットする | `None` |
| `os.setgroups(groups, /)` | 実行中のグループに関連付けられた従属グループ ID のリストを `groups` に設定する。`groups` はグ<br /><br />ループを特定する整数のリストとする。通常、この操作はスーパユーザーしか利用できない | `None` |
| `os.setpriority(which, who, priority)` | プログラムのスケジューリング優先度を設定する。`which` と `who` は `os.getpriority()` と同じ。<br /><br />`priority` は `-20` から `19` の整数値で、デフォルトの優先度は `0`。小さい数値ほど優先されるス<br /><br />ケジューリングとなる | `None` |

これらの関数は Windows では定義されない（呼び出すと `AttributeError` 例外が発生する）。

In [ ]:
import os
os.getuid(), os.geteuid(), os.getgid(), os.getegid(), os.getgroups(), os.getpriority(os.PRIO_PROCESS, 0)

(0, 0, 0, 0, [0], 0)

メモリ上でプロセスは次のような構成となる。

  * **テキストセグメント**: プログラムそのものが格納される読み出し専用の領域
  * **データセグメント**:
      * **静的領域**: 定数やグローバル変数を格納する領域
      * **ヒープ領域**: オブジェクトのように、実行するまで容量が未確定で一時的に必要とするデータを格納する領域
  * **スタックセグメント**: 関数とその中に定義された変数を下から順に格納し、上から実行され取り出されていく領域。**スタック領域**ともいう。

``` text
             メモリマップ
0xffffffff┏━━━━━━━━┓
          ┃                ┃
          ┃  スタック領域  ┃ スタックセグメント
      SP→┃                ┃
          ┣━━━━━━━━┫
                   ↓ 自動拡張

          ┏━━━━━━━━┓
          ┃共有ライブラリ１┃
          ┗━━━━━━━━┛
          ┏━━━━━━━━┓
          ┃共有ライブラリ２┃
          ┗━━━━━━━━┛

                   ↑ 自動拡張
          ┣━━━━━━━━┫
          ┃                ┃
          ┃   ヒープ領域   ┃
          ┃                ┃ データセグメント
          ┠────────┨
          ┃    静的領域    ┃
          ┣━━━━━━━━┫
          ┃                ┃
      PC→┃                ┃ テキストセグメント
          ┃                ┃ （読み出し専用）
         0┗━━━━━━━━┛

```

ヒープ領域が不足すると、Python は `MemoryError` 例外を送出する。スタック領域が不足すると、Python はエラー（stack overflow）を出してインタプリタの実行を中断する。

共有ライブラリは、ヒープ領域とスタック領域の間にある。どの番地に割り当てられるかは、システムに依存する。

また、メモリとは別に、 CPU 内にはレジスタと呼ばれる記憶装置があり、そこにはスタック領域のどこを見ているかを指す**スタックポインタ**（SP）と、プログラムのどこを実行しているかを指す**プログラムカウンタ**（PC）が置かれている。プロセスは、メモリ上の構成と SP、PC を最小セットする。このデータの最小セットを**コンテキスト**（context）と呼ぶ。

### マルチプロセス

同時に実行するプロセスが複数存在することを**マルチプロセス**（multiprocessing）という。マルチプロセスでも各プロセスにコンテキストが作られることに変わりはないから、各プロセスのメモリ空間は独立していてメモリ上のデータを共有することはない。

1 つの CPU コアは 1 つのプロセスだけが利用できる。CPU コアが足りていれば、マルチプロセスは並列処理される。CPU コアが不足すれば、マルチプロセスは並行処理される。1 つの CPU コアで実行されるプロセスが切り替わることは**コンテキストスイッチ**（context switching）と呼ばれる。具体的には、CPU コアがあるプロセスを実行している最中に処理を中断して現在のプロセスのコンテキストを特定のメモリ領域などに保存し、別のプロセスのコンテキストを読み込んで処理を再開するということが行われる。コンテキストスイッチによるプロセスの切り替えは、人間には認識できないほどに高速に行われるため、複数のプロセスが同時に実行されているように感じられる。

### スレッド

1 つのアプリケーション内で複数のプロセスを起動し並列処理を行うことも可能である。しかしながら、各プロセスは独立のメモリ空間が割り当てられるので、起動時にそれなりのオーバーヘッドが発生する。また、プロセス間でデータを交換するには特別な仕組みが必要となる。

そこで、1 つのプロセス内で並列処理を行う仕組みが作られた。このとき各 CPU コアを占有する一連の処理の流れのことを**スレッド**（thread）と呼ぶ。つまり、スレッドはプロセスより細かい CPU コア利用の最小単位とされるものである。プロセス起動時には必ず 1 個のスレッドが生成されるので、1 個のプロセスには最小限 1 個のスレッドが存在する。プロセス起動時に生成されるスレッドを**メインスレッド**と呼ぶ。メインスレッドから枝分かれするスレッドが存在することを**マルチスレッド**（multithread）といい、マルチスレッドでないものは**シングルスレッド**（singlethread）という。枝分かれしたスレッドからさらに枝分かれすることもできる。

``` text
            
  ┏━━━━━━━━━━┓     ┏━━━━━━━━━━━━━━┓
  ┃（シングルスレッド）┃     ┃     （マルチスレッド）     ┃
  ┃    ┌────┐    ┃     ┃        ┌────┐        ┃
  ┃    │ 処理Ａ │    ┃     ┃        │ 処理Ａ │        ┃
  ┃    └────┘    ┃     ┃        └────┘        ┃
  ┃         ↓         ┃     ┃         ↙        ↘         ┃
  ┃    ┌────┐    ┃     ┃ ┌────┐  ┌────┐ ┃
  ┃    │ 処理Ｂ │    ┃     ┃ │ 処理Ｂ │  │ 処理Ｄ │ ┃
  ┃    └────┘    ┃     ┃ └────┘  └────┘ ┃
  ┃         ↓         ┃     ┃      ↓            ↓      ┃
  ┃    ┌────┐    ┃     ┃ ┌────┐  ┌────┐ ┃
  ┃    │ 処理Ｃ │    ┃     ┃ │ 処理Ｃ │  │ 処理Ｅ │ ┃
  ┃    └────┘    ┃     ┃ └────┘  └────┘ ┃
  ┗━━━━━━━━━━┛     ┗━━━━━━━━━━━━━━┛
```

スレッドは、メモリのスタック領域とスタックポインタとプログラムカウンタから構成される。ヒープ領域と静的領域は同じものをスレッド間で共有する。つまり、処理を行う関数などの流れを作成して、変数などのデータは共有するという仕組みである。スレッドの構成はプロセスの構成より小さいため、スレッド生成時に発生するオーバーヘッドはプロセス生成時のものより少ない。また、共有するデータについてはスレッド間でデータ交換を行う必要がない。

### 論理コア

スレッドは CPU コアを占有するから、並列処理が可能なスレッドの数は物理的に存在するコア数に制限されるはずである。しかしながら、この制限を緩和する技術として**同時マルチスレッディング**（Simultaneous Multi-Threading; **SMT**）が存在する。ハイパースレッディング・テクノロジー（Hyper-Threading Technology）は、インテルの SMT 実装に対する同社の登録商標である。

SMT の概要は以下のとおり。物理的に存在する 1 個の CPU コアは複数の回路から構成され、それらは I/O 制御、整数演算、浮動小数点数演算などの役割を持つ。個々のスレッドは、CPU コア内の全ての回路を使うわけではないことが多い。たとえば、フィボナッチ数を調べるプログラムは整数演算ばかり行うスレッドを生成し、そのスレッドの処理中は浮動小数点数演算を行う回路が使われない。そこで、使用されない回路を他のスレッドに割り当て、2 つのスレッドが同時に実行するようにしたものが SMT である。各スレッドからは SMT は隠蔽され、2 つの CPU コアが動作しているかのように見える。このようにソフトウェア側から見える疑似的な CPU コアを**論理コア**と呼ぶ。SMT が有効な場合、1 スレッド - 1 論理コアという対応になる。

`os` モジュールは、論理コア数に関する関数を提供している:

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `os.cpu_count()` | システム全体で利用可能な論理コア数を返す。論理コア数を取得できなかった時 `None` を返す | `int` &vert; `None` |
| `os.process_cpu_count()` | 現在実行中のプロセスが使用できる論理コア数を返す。論理コア数を取得できなかった時 `None` を返す<br /><br />（Python 3.13 で追加） | `int` &vert; `None` |

システム全体で利用可能な論理コア数の全てを Python のプロセスで使用できるとは限らない。`os.process_cpu_count()` 関数は、CPU リソースを動的に確認でき、並列処理を行う際に、どの程度の並列を使うべきか調整することができる。

``` shell
>>> import os
>>> os.cpu_count()
12
>>> os.process_cpu_count()
12
```

また、標準ライブラリの `multiprocessing` パッケージも `cpu_count()` 関数を提供する。これも論理コア数を返すが、論理コア数を取得できなかった時 に` NotImplementedError` 例外を送出する。

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

### タスク

1 つのまとまった処理または命令を**タスク**（task）と呼ぶ。どの大きさでまとめるとタスクと言えるという定義はない。タスクは、文脈によって、スレッド内の命令の集合を指していたり、スレッドそのものを指していたり、プロセスを指していたり、プロセスの集合を指していたりする。

マルチプロセスは、**マルチタスク**（multitasking）とも呼ばれる。なお、マルチタスクは日本において NEC の登録商標である。

threading
---------

プロセスとスレッドは OS によって管理される。OS のスレッド周りの API に対する標準の Python インターフェースとしては、`_thread` モジュールと `threading` モジュールがある。`_thread` は OS の API に近い低レベルのインターフェースを提供していて、`threading` は `_thread` の上に構築された高レベルなインターフェースを提供している。よって、通常は `threading` を使う。

### インスタンス化

`threading` モジュールは、スレッドを `threading.Thread` オブジェクトとして扱う。コンストラクタは次のとおり:

``` python
threading.Thread(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
```

コンストラクタは常にキーワード引数を使って呼び出さなければならない。各引数は以下のとおり:

| 引数 | 意味 |
|:---|:---|
| `group` | `None` にする必要がある。この引数は将来 `ThreadGroup` クラスが実装されるときの拡張用に予約されている |
| `target` | インスタンスの `run()` メソッドによって起動される呼び出し可能オブジェクトを指定する。デフォルトでは何も呼び出さないことを示す `None`<br /><br /> になっている |
| `name` | スレッドの名前を指定する。値はインスタンスの `name` 属性で参照できる。デフォルトの `name` 属性は、`target` 引数が指定されない場合に<br /><br /> `Thread-N`（`N` は小さな 10 進数）となり、`target` 引数が指定された場合に `Thread-N (target)` という形式でユニークな名前が構成される |
| `args` | `target` を呼び出すときの位置引数をリストかタプルで指定する。デフォルトは `()` |
| `kwargs` | `target` を呼び出すときのキーワード引数を辞書で指定する。デフォルトは `{}` |
| `daemon` | このスレッドがデーモン（メインメモリ上に常駐してバックグラウンドで動作するプログラム）であるか否かを示すブール値を設定する。値はイン<br /><br />スタンスの `daemon` 属性で参照できる。`None`（デフォルト）の場合、`daemon` 属性は現在のスレッドから継承される |

`threading.Thread` オブジェクトの識別には、`name` 属性（読み書き可能なプロパティ）または `ident` 属性（読み取り専用プロパティ）が使える。`ident` 属性はスレッドが開始されていなければ `None` で、開始されていれば非ゼロの整数が格納されている。

`threading.Thread` オブジェクトのメソッドは次のとおり:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `start()` | スレッドを開始し、`run()` メソッドを実行する。このメソッドはオブジェクトあたり一度だけ呼び出すことができる | `None` |
| `run()` | このスレッド内で `target` を呼び出す | `None` |
| `join(timeout=None)` | スレッドが終了するまで待機する。`timeout` に浮動小数点数を指定した場合は、このメソッドは `timeout` 秒でタイムアウト<br /><br />する（次のコードに進む）。現在のスレッドに対してこのメソッドを呼び出そうとすると `RuntimeError` 例外を送出する | `None` |
| `is_alive()` | スレッドが動作中の場合に `True` を返し、そうでない場合に `False` を返す。`join()` がタイムアウトしたかどうかはこのメ<br /><br />ソッドで確認できる | `bool` |

`threading` モジュールは、ロードされると暗黙のうちにメインスレッド（Python インタプリタが起動したスレッド）に対応するオブジェクトを作成する。このオブジェクトは、`threading.Thread` クラスを継承する `threading._MainThread` クラスのインスタンスであり、`name` 属性が `'MainThread'` になっている。

``` python
threading.current_thread()
```

この関数は、関数を呼び出している処理のスレッドに対応する `threading.Thread` オブジェクトを返す。関数を呼び出している処理のスレッドが `threading` モジュールで生成したものでない場合、限定的な機能しかもたないダミースレッドオブジェクトを返す。

``` python
threading.enumerate()
```

この関数は、現在、アクティブな `threading.Thread` オブジェクト全てのリストを返す。リストには、終了したスレッドとまだ開始していないスレッドは入らない。しかし、メインスレッドは、たとえ終了しても、常に結果に含まれる。

次のコードは、1 秒ごとに 3 回まで `print()` 関数で出力する `run()` 関数を 2 つのスレッドで実行する。

In [ ]:
import threading
import time


def run(base):
    for i in range(3):
        time.sleep(1.0)
        print("{}: {}".format(threading.current_thread().name, base + i))


def main(*args):
    print(f"started at {time.strftime('%X')}")
    for t in args:
        t.start()
    for t in args:
        t.join()
    print(f"finished at {time.strftime('%X')}")


if __name__ == "__main__":
    t1 = threading.Thread(target=run, name="t1", args=(10,))
    t2 = threading.Thread(target=run, name="t2", args=(20,))
    main(t1, t2)

started at 02:39:45
t1: 10
t2: 20
t1: 11
t2: 21
t1: 12
t2: 22
finished at 02:39:48


名前が `t1` と `t2` の 2 つのスレッドが交替で実行されていることを確認できる。2 つのスレッドを開始してから終了するまでの時間が 3 秒なので、並列処理が行われている。

次のコードは、リストを 20 万回 `append()` する `run()` 関数を 2 個のスレッドで実行し、その後に同関数を 1 個のスレッドで実行する。

In [ ]:
import threading
import time


def run():
    a = []
    for i in range(20000000):
        a.append(0)


def main(*args):
    print(f"started at {time.strftime('%X')}")
    for t in args:
        t.start()
    for t in args:
        t.join()
    print(f"finished at {time.strftime('%X')}")


if __name__ == "__main__":
    t1 = threading.Thread(target=run)
    t2 = threading.Thread(target=run)
    t3 = threading.Thread(target=run)
    print("2スレッド-----------")
    main(t1, t2)
    print("1スレッド-----------")
    main(t3)

2スレッド-----------
started at 02:39:48
finished at 02:39:54
1スレッド-----------
started at 02:39:54
finished at 02:39:57


2 スレッドの場合の処理時間が 1 スレッドの場合の 2 倍になっていることから、2 スレッドは並列処理されていないことがわかる。

2 つのコードの実行結果の違いはどのような理由によるものであろうか。

### GIL

実は、 CPython では、マルチスレッドがあっても CPU コアを占有できるのはロックを保持する単一のスレッドに制限される。このロックを**グローバルインタプリタロック**（Global Interpreter Lock）、略して GIL（ギル）と呼ぶ。GIL の獲得と解放により、動作するスレッドが切り替わる。つまり、**CPython のマルチスレッドは並行処理となる**。ただし、GIL は Python で書かれた処理についての制限であり、モジュールが利用する C などで書かれた処理は GIL の制限を受けず、並列処理が可能である。

CPython に GIL が存在する理由は、C における「データ競合」を回避するためである。C では、1 つのプロセス内の 2 つ以上のスレッドがメモリ上の同じ場所に同時にアクセスすることによってプログラムが C の規格で未定義の振る舞いをする現象があり、この現象を**データ競合**（data race）と呼ぶ。データ競合はソフトウェアセキュリティの脆弱性につながる可能性がある。データ競合を回避するには、メモリへのアクセスを制御（ロック）する必要がある。ところが、この実装では、ロックが必須なメモリアクセスが多すぎて、パフォーマンスが大きく低下することがわかっている。そこで、個別のメモリアクセスを制御するのではなく、スレッドの動作全体を制御する GIL が導入された。

データ競合は C に限らない現象であるが、その定義や扱いはプログラミング言語ごとに異なる。このため、GIL の存在は Python の言語仕様として要求されていない。実際、Java で開発された Python の処理系（Jython）には GIL は存在しない。しかしながら、最も普及している Python の処理系は CPython なので、事実上マルチスレッド Python プログラムは並行処理と考えなければならない。

シングルスレッドでは、データ競合は発生しないので、GIL の解放は起こらないようになっている。つまり、シングルスレッドの場合、スレッドは何事もなく最後まで実行される。

マルチスレッドの場合、GIL の獲得と解放がおおむね以下のように行われる。

  * あるスレッドが GIL を獲得すると、それだけが CPU コアを占有する。他のスレッドは（実行中であっても）休止する。
  * GIL を保持するスレッドは、I/O 操作が発生したら、自発的に全てのスレッドに通知を行う。これを受けて、他のスレッドが GIL を獲得する。
  * 休止中のスレッドは、タイムアウト（デフォルトで 5 ミリ秒後）になったら C のグローバル変数 `gil_drop_request` の値（デフォルト値は 0）を 1 に設定する。`gil_drop_request` の値が 1 の場合、GIL を保持するスレッドは全てのスレッドに通知を行う。これを受けて他のスレッドが GIL を獲得すると、GIL を解放したスレッドに通知し、`gil_drop_request` の値は 0 に戻される。
  * スレッドの切り替え時にはコンテキストスイッチが行われるので、スレッドが GIL を解放した後、再び GIL を獲得して実行する場合、前回の続きから再開する。

`sys` モジュールは、GIL のタイムアウト時間を取得、設定する関数を提供している。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `sys.getswitchinterval()` | GIL のタイムアウト時間を返す | `float` |
| `sys.setswitchinterval(interval)` | GIL のタイムアウト時間を `interval`（浮動小数点数）に指定する | `None` |

In [ ]:
import sys
sys.getswitchinterval()

0.005

GIL はタイムアウトしていなければ「I/O 操作発生」によってのみ解放されるわけであるが、I/O 操作を行う処理のことを **I/O バウンド**（I/O bound）と呼ぶ。システム時刻の問い合わせ、標準入出力やファイルの読み書き、ネットワーク通信はすべて I/O バウンドな処理である。

一方、I/O 操作を伴わない処理（論理演算や整数演算、浮動小数点数演算など）のことは **CPU バウンド**（CPU-bound）と呼ぶ。Python オブジェクトの操作は CPU バウンドな処理である。

2 種類のマルチスレッド処理で GIL の影響の違いは次のように整理できる。

  * I/O バウンドな処理:  
I/O 操作が行われるたびにスレッドが切り替わる。遅いネットワーク通信などを待たずに他の処理に移ることができるため効率がよい。
  * CPU バウンドな処理:  
計算の途中でもタイムアウトによりスレッドが切り替わるため、余計に計算コストがかかり遅くなる。

`threading.Thread` インスタンス化の例として示した 2 つのコードのうち、1 番目のものについては、`run()` 関数内での `time.sleep()` 関数の実行と `print()` 関数の実行はともに I/O バウンドであり、I/O 操作が発生して下図のようにスレッドが切り替わるため、2 つのスレッドが交替で実行され、スリープ時間で処理時間が 2 倍になることはなかったわけである。つまり、I/O バウンドな処理では並列処理が実現される。この並列処理は CPU コア数以上にスケールできる（CPU コアは常に 1 個しか利用されない）。

``` text
           ┌─────────────────────────────────────────────────────┐
           │      ┌───────┐                                            ┌─────┐                        │
t1 thread: │run() │ time.sleep() │I/O 操作 ………………[スリープ]……………… │ print()  │I/O 操作   （以下省略） │
           │      └───────┘              タイムアウト  タイムアウト    └─────┘                        │
           └─────────────────────────────────────────────────────┘
                                    ↓      ↑           ↓    ↑      ↓     ↑                ↓      ↑
           ┌─────────────────────────────────────────────────────┐
           │              ┌───────┐          タイムアウト  タイムアウト   ┌─────┐                     │
t2 thread: │run()         │ time.sleep() │I/O 操作 …………[スリープ]………………│ print()  │I/O 操作 （以下省略）│
           │              └───────┘                                       └─────┘                     │
           └─────────────────────────────────────────────────────┘
```

2 番目のコードについては、`run()` 関数を実行するスレッドは CPU バウンドであり、2 スレッドの場合はタイムアウトでスレッドが切り替わる並行処理が行わる。このため、1 スレッドだけで計算する場合より処理時間が 2 倍以上長くなったのである。

では、I/O バウンドなスレッドと、CPU バウンドなスレッドが同時に走っている場合はどうなるのであろうか。スレッドは優先度を持たないため、どちらが先に GIL を獲得するのかはわからないのであるが、獲得順によって処理の効率は次のように異なる:

  * I/O バウンドなスレッドが先に GIL を獲得する場合、I/O 操作発生で CPU バウンドなスレッドに切り替わり、I/O の待機中に CPU バウンドな処理が進行することになるので、効率が良い。
  * CPU バウンドなスレッドが先に GIL を獲得する場合、タイムアウトになるまで CPU バウンドな処理が進行し、その後に I/O バウンドなスレッドに切り替わって I/O 待ちに入るということになるので、I/O 待ちの待機時間の分だけ実行時間が長引くことになる。

2 つ目のケースように非効率な形で GIL が獲得される現象を **Convoy Effect** という。Convoy Effect が発生することを防ぐ方法はない。

### GIL の無効化

Python 3.13 では、実験的な機能として、GIL を無効化した CPython をビルドできるようになった。

GIL を無効化したマルチスレッドなら、CPU バウンドなスレッドも並列処理が可能となるから、マルチコア CPU を活かすことができ、パフォーマンス向上が期待できる。

しかし、GIL の無効化は、以下の問題があることに注意する。

  * GIL 除去によるパフォーマンス低下を最低限にするための手法が導入されているものの、全てのケースでマルチスレッドのパフォーマンスが改善されるわけではない。
  * 「I/O 操作発生」によるスレッド切り替えが行われなくなるので、I/O バウンドな処理の並列化は CPU コア数以上にスケールできなくなる。
  * シングルスレッドでは逆に性能が低下する。
  * GIL に依存するライブラリでは不具合が生じる。

GIL を無効化した CPython バイナリを**フリースレッドバイナリ**（free-threaded binary）と呼ぶ。

公式の Windows 用 Python インストーラーはフリースレッドバイナリを含んでおり、「Customize installation」を選択して進むと表示される「Advanced Options」の中の「Download free-threaded binaries」をチェックすることで、フリースレッドバイナリがインストールされる。フリースレッドバイナリは、`python3.13t` のように `t` を付加したファイル名でインストールされるので、通常の Python と共存してインストールできる。

Python ランチャーでフリースレッドバイナリを指定するには、次のように実行する:

``` shell
py -3.13t
```

フリースレッドバイナリを使う場合でも、Python 言語の機能は通常の Python と全く同じであり、特別な構文が導入されるわけではない。

フリースレッドバイナリでも、環境変数 `PYTHON_GIL` を `1` に設定すると GIL が有効になる。`PYTHON_GIL` を `0` に設定すると GIL が無効になる。

フリースレッドバイナリでは、次の関数が使える。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `sys._is_gil_enabled()` | GIL が有効になっている場合は `True` を返し、無効になっている場合は `False` を返す | `bool` |

### デーモンスレッド

コンストラクタの `daemon` 引数またはインスタンスの `daemon` プロパティを `True` に設定したスレッドはデーモンとして稼働する。デーモンスレッドは、メインスレッドを含めてデーモンでないスレッドがすべて終了すると停止する。

次のコードでは、`t2` スレッドで実行される `run2()` 関数はグローバル変数 `request`に値を代入して終了する。`t1` スレッドはデーモンとして稼働し、そこで実行される `run1()` 関数は無限ループを実行する。その無限ループの中で、`request` 変数に値が代入されたらその値を出力する。

``` python
import time
import threading

request = None


def run1():
    global request
    while True:
        if request is None:
            time.sleep(0.2)
            print("{}: running...".format(threading.current_thread().name))
        else:
            print("{}: {} is received".format(threading.current_thread().name, request))
            request = None


def run2():
    global request
    time.sleep(0.5)
    request = "hoge"
    time.sleep(0.2)
    print("{}: end".format(threading.current_thread().name))


def main():
    t1 = threading.Thread(target=run1, name="t1", daemon=True)
    t2 = threading.Thread(target=run2, name="t2")
    t1.start()
    t2.start()
    t2.join()


if __name__ == "__main__":
    main()
    print("{}: end".format(threading.current_thread().name))
```

このコードの実行結果は以下のようになる。

``` shell
t1: running...
t1: running...
t1: running...
t1: hoge is received
t2: end
MainThread: end
```

`run1()` 関数の無限ループは、break 文がないにもかかわらずメインスレッドの終了後に停止するため、プログラムはちゃんと終了することがわかる。なお、`main()` 関数の最後に `t1.join()` を加えると、`t1.join()` は `run1()` 関数の終了を待つのでプログラムはいつまでたっても終了しなくなることに注意する。

### 競合状態

マルチスレッドの実行順序やタイミングによって異なる結果が生まれる状態を**競合状態**（race condition）という。競合状態とデータ競合は、似て非なる概念である。GIL の存在によって Python ではデータ競合は発生しないが、競合状態は起こり得る。

たとえば、次のコードでは、`counter()` 関数を動かすスレッドを 3 つ作成している。`counter()` 関数では、引数に渡された辞書 `data` に対して `data['n']` の値（初期値 0）を 1 増加する処理を行うが、3 つのスレッドが終了した時点で `data['n']` の値は 3 になるとは限らない。

In [ ]:
import threading
import time


def counter(data):
    n = data['n']
    time.sleep(0.2)
    data['n'] = n + 1
    print(f"{threading.current_thread().name}: {data['n']=}")


def main():
    data = {'n': 0}
    threads = [threading.Thread(target=counter, name=f"t{i}", args=(data,)) for i in range(3)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(f"{threading.current_thread().name}: {data['n']=}")


if __name__ == "__main__":
    main()

t0: data['n']=1
t1: data['n']=1
t2: data['n']=1
MainThread: data['n']=1


最初に GIL を獲得したスレッドで `counter()` 関数内のローカル変数 `n` に `data['n']` の初期値 0 を代入し、直後に `time.sleep()` を呼び出す。この時点でスレッドが切り替わり、別のスレッドでもローカル変数 `n` に `data['n']` の初期値 0 が代入される。もし `time.sleep()` の呼び出しまでに 3 つのスレッドが 1 回ずつ実行されていれば、各スレッドでローカル変数 `n` の値は 0 になっている。この場合、その後のコンテキストスイッチにより、今度は各スレッドで `data['n']` に `0 + 1` の計算結果値が代入されることになるため、3 つのスレッドが終了した時点で `data['n']` の値は 3 になるとは限らないわけである（1 になるとも 2 になるとも限らない）。

競合状態での結果は、「プログラマーがそう書いたからそう動いている」だけであり、プログラミング言語で未定義の動作が起こっているわけではない。ただ、その結果がプログラマーの意図に反するものであればバグと言える。

### スレッドの同期

複数の処理について、処理の進行を待ち合わせることを**同期**（synchronous）という。同期でないものは**非同期**（asynchronous）という。マルチスレッドまたはマルチプロセスのプログラミングにおいて、異なるスレッドやプロセス間で同期を取るための制御機構を**同期プリミティブ**（synchronization primitive）という。

マルチスレッドの競合状態を、複数人でオールを漕いでボートが進む状態に例えるなら、漕ぎ手がスレッド、ボートがプログラムであり、漕ぎ手のタイミングを合わせないとボートが正しい方向に進まないように、スレッド間で同期を取らないとプログラムが正しく動作しないのである（画像は[いらすとや](https://www.irasutoya.com/2012/07/blog-post_3963.html)より）。同期プリミティブは、漕ぎ手のタイミング合わせに使われる掛け声や笛に相当する。

<img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhkr3KmVJffiUV0pebpKjpZLEVi7KVbmRbH7y4BBFZHkee5Yq8TtT7kDQxdB8VnrJ6wMFOsuwB1Lm9ER2GilJ0XIb28mHwxAz3sOSJLz48r9uGwd5BjoQcUoLf3FC1sTxSQ7-uA8Knx9D8/s800/olympic21_boat_4.png" width="400">

`threading` モジュールは、以下の同期プリミティブをサポートする。

  * ミューテックス（ロック）
  * セマフォ
  * イベント（Event）
  * コンディション（Condition）── 一般には条件変数（Condition Variables）と呼ばれる
  * バリア（Barrier）

### 直列化とミューテックス

スレッドを列に並ばせるように待機させ、一つずつ順番に実行させるような同期の取り方を**逐次化**あるいは**直列化**（serialize）と呼ぶ。共有データを扱う場面での競合状態では、直列化を行わないとバグが発生する可能性がある。

一方、あるスレッドが共有データなどの資源を利用している間は、他のスレッドが同じものにアクセスすることを制限もしくは禁止する仕組みのことを**排他制御**という。とくに、1 つのスレッドが資源を占有し、他のスレッドからのアクセスを禁止するという排他制御の方式は**ロック**（lock）と呼ばれる。GIL もロックの一種である。ロック方式は、代表的な排他制御なので、排他制御を意味する英語 mutual exclusion を略した形で**ミューテックス**（mutex）とも呼ばれる。ミューテックスは、直列化の手法として利用される。

``` python
threading.Lock()
```

これはロックを返す。ただし、実際には `threading.Lock` というクラスが定義されているわけではなく、これは `_thread.get_ident()` 関数の別名であることに注意すること。

ロックのメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `acquire(blocking=True, timeout=-1)` | ロックの獲得を試み、実際に獲得すると `True` を、獲得できなかったとき `False` を返す。他のスレッドが既<br /><br />にロックを獲得している場合、`blocking` と `timeout` の指定によってメソッドの挙動が異なる。`blocking`<br /><br /> が `True`（デフォルト）なら、ロックが解放されるまで待機する。`blocking` が `False` なら、直ちに `False` を<br /><br />返し待機しない。`timeout` に正の浮動小数点が設定された場合、`timeout` 秒だけ待機する。`blocking` が<br /><br /> `False` の場合に `timeout` を指定することは禁止される | `bool` |
| `release()` | ロックを解放する。これはロックを獲得したスレッドだけでなく、任意のスレッドから呼ぶことができる | `None` |

ロックはコンテキストマネージャーとして使用できる。`acquire()` メソッドは with ブロックに入るときに呼び出され、`release()` メソッドはブロックを終了するときに呼び出される。したがって、次のコード片

``` python
with some_lock:
    # do something...
```

これは、以下と同じ。

``` python
some_lock.acquire()
try:
    # do something...
finally:
    some_lock.release()
```

次のコードは、競合状態のコード例にロックを追加したものである。

In [ ]:
import threading
import time


def counter(data, lock):
    with lock:  # ロックの獲得と解放
        n = data['n']
        time.sleep(0.2)
        data['n'] = n + 1
        print(f"{threading.current_thread().name}: {data['n']=}")


def main():
    data = {'n': 0}
    lock = threading.Lock()  # ロックを作成
    threads = [threading.Thread(target=counter, name=f"t{i}", args=(data, lock)) for i in range(3)]  # ロックを渡す
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(f"{threading.current_thread().name}: {data['n']=}")


if __name__ == "__main__":
    main()

t0: data['n']=1
t1: data['n']=2
t2: data['n']=3
MainThread: data['n']=3


`data['n']` の値は、各スレッドが動かす `counter()` 関数により確実に 1 増加され、3 つのスレッドが終了した時点で 3 になる。この実行結果から、`counter()` 関数内の with 文でロックを獲得したスレッドが with 文の本体を処理している間、スレッド切り替えが行われず、このため `data['n']` に対する操作が直列化されていることがわかる。

一方、ロックの解放を待つスレッドは、待機中に別の処理ができれば効率的である。この場合、ロックの `acquire()` メソッドを引数 `blocking=False` として呼び出すと、ロックを獲得できなければ待機せず直ちに `False` を返すことを利用する。

次のコードは、前のコードに以下の変更を加えたものである。

  * `counter()` 関数は、無限の while ループを使って、ロックを獲得できれば `data['n']` の値を 1 増加してロック解放後にループを終了するが、ロックを獲得できない間は別の作業（時刻の表示）をする。

In [ ]:
import threading
import time


def counter(data, lock):
    while True:
        if lock.acquire(blocking=False):
            print(f"{threading.current_thread().name}: ロックを獲得した")
            n = data['n']
            time.sleep(0.2)
            data['n'] = n + 1
            print(f"{threading.current_thread().name}: {data['n']=}")
            lock.release()
            break
        else:
            print(f"{threading.current_thread().name}: {time.strftime('%X')}")
            time.sleep(0.2)


def main():
    data = {'n': 0}
    lock = threading.Lock()  # ロックを作成
    threads = [threading.Thread(target=counter, name=f"t{i}", args=(data, lock)) for i in range(3)]  # ロックを渡す
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(f"{threading.current_thread().name}: {data['n']=}")


if __name__ == "__main__":
    main()

t0: ロックを獲得した
t1: 02:39:57t2: 02:39:57

t0: data['n']=1
t2: ロックを獲得した
t1: 02:39:58
t2: data['n']=2
t1: ロックを獲得した
t1: data['n']=3
MainThread: data['n']=3


実行結果から、あるスレッドがロックを保持している間も、他のスレッドは待機せずに時刻表示処理を行っていることがわかる。このように、ロックの獲得で待機しないことを**ノンブロッキング**（non-blocking）であるという。

### デッドロック

**デッドロック**（deadlock）とは、2 つ以上のスレッド（あるいはプロセス）が互いに待機しあってしまい、結果としてどの処理も先に進めなくなってしまうことをいう。

以下のコードは、デッドロックを発生させる簡単な例である。

``` python
import time
import threading


def worker1(data, lock1, lock2):
    with lock1:
        n = data['n']
        time.sleep(0.2)
        with lock2:
            m = data['m']
            data["n"] = n - m
            print(f"{threading.current_thread().name}: {data['n']=}")


def worker2(data, lock1, lock2):
    with lock2:
        m = data['m']
        time.sleep(0.2)
        with lock1:
            n = data['n']
            data['m'] = m + n
            print(f"{threading.current_thread().name}: {data['m']=}")


def main():
    data = {'n': 0, 'm': 0}
    lock1 = threading.Lock()
    lock2 = threading.Lock()
    threads = [
        threading.Thread(target=worker1, name="t1", args=(data, lock1, lock2)),
        threading.Thread(target=worker2, name="t2", args=(data, lock1, lock2)),
    ]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(f"{threading.current_thread().name}: {data['n']=}")


if __name__ == "__main__":
    main()
```

このコードでは、2 つの共有データの要素 `data['n']` と `data['m']` に対するアクセスを別々に制御するため、2 つのロック `lock1` と `lock2` を作成する。`worker1()` 関数は `lock1` を獲得した後、`lock2` を獲得しようとする。同様に、`worker2()` 関数は `lock2` を獲得した後、`lock1` を獲得しようとする。ロックの獲得が循環的になっているため、いつまでも条件を満たせず、デッドロックが発生する。

この問題は、**食事する哲学者の問題**として知られる。円卓に麺と箸 1 本が交互に並べられた状況で、席に着いた哲学者はまず左側の箸を取ってから次に右側の箸を取ってよいという制約があると仮定する。全員が同時に左側の箸を取ってしまうと、右側の箸が使えず、いつまでたっても食事ができないことになる（画像は https://docs.oracle.com/cd/E19205-01/821-2500/gepdy/index.html より引用）。

![](https://docs.oracle.com/cd/E19205-01/821-2500/images/figure2.gif)

デッドロックを検出するには、タイムアウトを設定する。つまり、ロックの `acquire()` メソッド呼び出し時に `timeout` を設定し、メソッドの戻り値が `False` なら例外を送出する。

``` python
try:
    acquired = lock1.acquire(timeout=3)
    if not acquired:
        raise TimeoutError("{}: Deadlock detected".format(threading.current_thread().name))
    # do something
finally:
    lock1.release()
```

ただし、ロック獲得にタイムアウトを設定する場合、同時にノンブロッキングには設定できないことに注意する。

### 再入可能ロック

デッドロックは、同じスレッドで同じロックを複数回獲得しようとする場合にも発生する。

たとえば、次のコードでは `worker()` 関数が再帰関数であり、呼び出すと引数 `times` を増やしながら再帰呼び出しが行われ、`times` が 3 以上になったら終了することを意図している。

``` python
import time
import threading


def worker(data, times, lock):
    print(f"{times=}")
    if times >= 3:
        return None
    else:
        with lock:
            n = data['n']
            time.sleep(0.2)
            data['n'] = n + 1
            worker(data, times + 1, lock)


def main():
    data = {'n': 0}
    lock = threading.Lock()
    thread = threading.Thread(target=worker, args=(data, 0, lock))
    thread.start()
    thread.join()
    print(f"{data['n']=}")


if __name__ == "__main__":
    main()
```

ところが、このコードを実行すると、次のように出力されたまま先に進まない:

``` text
times=0
times=1
```

1 回目の再帰呼び出しから先に進めないというデッドロックが起こる。これは、`worker()` の with 文の中でロックが解放されないまま再帰呼び出しがあり、そこでロックを獲得できず待機するからである。

この場合は、次の関数により得られる**再入可能ロック**（reentrant lock）を使用すればデッドロックを回避できる:

``` python
threading.RLock()
```

通常のロックと再入可能ロックの違いは、以下の点だけである。

  * いったんスレッドが再入可能ロックを獲得すると、同じスレッドはブロックされずにもう一度それを獲得できる。
  * そのスレッドは獲得した回数だけ再入可能ロックを解放しなければならない。

次のコードは、上記のコードを、再入可能ロックを使用するように変更したものである。

In [ ]:
import threading
import time


def worker(data, times, lock):
    print(f"{times=}")
    if times >= 3:
        return None
    else:
        with lock:
            n = data['n']
            time.sleep(0.2)
            data['n'] = n + 1
            worker(data, times + 1, lock)


def main():
    data = {'n': 0}
    lock = threading.RLock()
    thread = threading.Thread(target=worker, args=(data, 0, lock))
    thread.start()
    thread.join()
    print(f"{data['n']=}")


if __name__ == "__main__":
    main()

times=0
times=1
times=2
times=3
data['n']=3


### セマフォ

ミューテックスより緩く、最大〇個のスレッドが共有資源にアクセスして良いとする排他制御の方式を**セマフォ**（semaphore）という。セマフォは、内部に空き数を示すカウンターを持つ。

セマフォでは直列化を実現できない。セマフォは、メモリ不足の回避やサーバーの負荷分散などの理由で、同時実行数に制限が必要な場合に使用される。

`threading` は、2 種類のセマフォをサポートする。1 つは通常のセマフォであり、もう 1 つは**有限セマフォ**（bounded semaphore）である。これらはそれぞれ `threading.Semaphore` クラスと `threading.BoundedSemaphore` クラスで実装される。インスタンス化は次のとおり:

``` python
threading.Semaphore(value=1)
threading.BoundedSemaphore(value=1)
```

`value` 引数は内部カウンターの初期値である。`value` を指定しない場合、デフォルトの値は 1 になる。セマフォは、ロックと同様に `acquire()` メソッドと `release()` メソッドを持ち、コンテキストマネージャーとして使用できる。

`acquire()` メソッドを呼び出すとカンターが 1 つ減少し、`release()` メソッドを呼び出すとカウンタが 1 つ増加する。カウンターが 0 の場合に `acquire()` メソッドを呼び出すと、カウンターが 1 以上になるまで待機する。この動作は、`acquire()` メソッドの引数 `blocking` と `timeout` で変更できる。

`release()` メソッドは引数 `n` として整数値を渡すことができ、内部カウンターを `n` だけ増加することもできる（`n` のデフォルトの値が 1 とされる）。通常のセマフォの `release()` メソッドの場合、何度呼び出してもカウンターは増加し続け初期値を上回ることができる。これに対して、有限セマフォの `release()` メソッドの場合、カウンターが初期値を上回ると `ValueError` 例外を送出する。

`acquire()` メソッドと `release()` メソッドを直接使う代わりに with 文でセマフォを使う限り、セマフォの内部カウンターが初期値を上回ることは考えにくいのであるが、カウンターの上限を確実に初期値に制限したい場合は、有限セマフォを選択するとよい。

次のコードは、Web サーバーから複数のファイルをダウンロードする処理をマルチスレッド化する例である。最大同時接続数を 2 とするために有限セマフォを利用している。

In [ ]:
import random
import threading
import time


def download(url, semaphore):
    with semaphore:
        print(f"{url} からのダウンロード開始")
        time.sleep(random.uniform(0.2, 1.0))  # ダウンロードの処理時間をシミュレート
        print(f"{url} からのダウンロード終了")


def main():
    semaphore = threading.BoundedSemaphore(2)
    threads = [threading.Thread(target=download, args=(f"url{i}", semaphore)) for i in range(5)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


if __name__ == "__main__":
    main()

url0 からのダウンロード開始
url1 からのダウンロード開始
url1 からのダウンロード終了
url2 からのダウンロード開始
url2 からのダウンロード終了
url3 からのダウンロード開始
url0 からのダウンロード終了
url4 からのダウンロード開始
url4 からのダウンロード終了
url3 からのダウンロード終了


### イベントとスレッド間通信

**イベント**は、特定の条件が満たされたことをスレッド間で**通知する**（notify）ために使用される。たとえば、あるタスクが完了したことを他のタスクに知らせたい場合に使用される。このようにスレッド間で通知が行われることを**スレッド間通信**と呼ぶ。

イベントは内部にフラグを持ち、このフラグがセットされたりクリアされたりする。同期を取りたいスレッド間でイベントを共有し、あるスレッドにより内部フラグがセットされることで他のスレッドは通知を受ける。このようにスレッド間通信と言っても、その仕組みは単純で、各スレッドからアクセスできる変数（グローバル変数や引数など）を介して値を操作したり確認したりしているだけである。

``` python
threading.Event()
```

これは、内部フラグの値が `False` であるイベントオブジェクトを返す。イベントオブジェクトは、次のメソッドを持つ:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `set()` | 内部フラグの値を `True` にセットする | `None` |
| `clear()` | 内部フラグの値を `False` にリセットする | `None` |
| `wait(timeout=None)` | 内部フラグがセットされない間は待機する。内部フラグがセットされると待機を解除し、プログラムの実行を続ける。`timeout` に<br /><br />浮動小数点数を指定した場合は、このメソッドは `timeout` 秒でタイムアウトする（プログラムの実行を続ける）。戻り値は、内部フ<br /><br />ラグがセットされたために返された場合は `True`、`timeout` が指定されてタイムアウトした場合は `False` | `bool` |
| `is_set()` | 内部フラグが `True` にセットされているとき `True` を返す | `bool` |

`set()` でいったん内部フラグが `True` になると、スレッドが `wait()` を呼び出しても待機しなくなる。`clear()` で内部フラグをリセットすれば、`wait()` を呼び出したスレッドは待機するようになる。

次のコードは、イベントの使用例である。`worker()` 関数が動くスレッドたちは、`wait()` を呼び出すと待機しイベントを監視する。`event_trigger()` 関数が動くスレッドが `set()` で内部フラグをセットすると、`worker()` 関数が動くスレッドたちは同時に開始される。

In [ ]:
import threading
import time


def worker(event):
    event.wait()
    print(f"{threading.current_thread().name}: start")
    time.sleep(0.2)
    print(f"{threading.current_thread().name}: end")


def event_trigger(event):
    print(f"{threading.current_thread().name}: start")
    time.sleep(0.5)
    print(f"{threading.current_thread().name}: end")
    event.set()


def main():
    event = threading.Event()
    threads = [
        threading.Thread(target=worker, name="t1", args=(event,)),
        threading.Thread(target=worker, name="t2", args=(event,)),
        threading.Thread(target=event_trigger, name="t3", args=(event,)),
    ]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


if __name__ == "__main__":
    main()

t3: start
t3: end
t2: start
t1: start
t2: end
t1: end


イベントは、あるスレッドの処理を完了させてから、その結果を利用して別のスレッドで処理を行いたい場合に便利である。ミューテックスではスレッドの順番を指定できないことに注意する。

### コンディション

**コンディション**も、イベントと同様に特定の条件が満たされたときにスレッドの実行を再開するための同期プリミティブである。イベントと同様に `wait()` を呼び出したスレッドたちが待機し、通知を受けると起こされる。イベントと違って、コンディションは内部にロックを持つ。スレッドの実行再開は、ロック方式の排他制御が行われて直列化される。

``` python
threading.Condition(lock=None)
```

これはコンディションオブジェクトを返す。デフォルトでは、内部ロックとして再入可能ロックが新たに自動的に作成される。既存のロックをコンストラクタの引数として渡すこともでき、これにより複数のコンディションで同じロックを共有できる。

コンディションオブジェクトは、次のメソッドを持つ:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `acquire(*args)` | 内部ロックの `acquire()` メソッドを呼び出し、その戻り値を返す | `bool` |
| `release()` | 内部ロックの `release()` メソッドを呼び出す | `None` |
| `wait(timeout=None)` | 通知を受けるか、タイムアウトするまで待機する。このメソッドはいったんロックを解放し、一度スレッドが起こされると、再度<br /><br />ロックを獲得して処理を戻す。与えられた `timeout` が過ぎていなければ返り値は `True` となる。タイムアウトした場合には<br /><br /> `False` を返す | `bool` |
| `notify(n=1)` | 待機中スレッドのうち `n` 個のスレッドを起こす。`n` のデフォルト値は `1` | `None` |
| `notify_all()` | 全ての待機中スレッドを起こす | `None` |

コンディションは、ロックと同様にコンテキストマネージャーとして使用できる。

`acquire()` と `release()` 以外のメソッドは、呼び出し側のスレッドがロックを獲得していないときに呼び出すと `RuntimeError` が送出される。

次のコードは、イベントの使用例をコンディションで書き直した例である。

In [ ]:
import threading
import time


def worker(condition):
    with condition:
        condition.wait()
        print(f"{threading.current_thread().name}: start")
        time.sleep(0.2)
        print(f"{threading.current_thread().name}: end")


def event_trigger(condition):
    with condition:
        print(f"{threading.current_thread().name}: start")
        time.sleep(0.5)
        print(f"{threading.current_thread().name}: end")
        condition.notify_all()


def main():
    condition = threading.Condition()
    threads = [
        threading.Thread(target=worker, name="t1", args=(condition,)),
        threading.Thread(target=worker, name="t2", args=(condition,)),
        threading.Thread(target=event_trigger, name="t3", args=(condition,)),
    ]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


if __name__ == "__main__":
    main()

t3: start
t3: end
t2: start
t2: end
t1: start
t1: end


`t1` スレッドは、`worker()` 関数内の `with condition` でロックを獲得して `wait()` を呼び出すと、いったんロックを解放して待機する。`t2` スレッドも同様である。`t3` スレッドは、`event_trigger()` 関数内の `with condition` でロックを獲得すると、ロックを保持したまま処理を完了し、最後に `notify_all()` を呼び出す。これにより `t1` スレッドが起こされると、再度ロックを獲得して続きの処理を行う。`t2` スレッドも同様である。`t1` スレッドと `t2` スレッドがロックを伴って動作するため、直列化される。

コンディションは、共有データを扱う場面での競合状態で、スレッドの順序付けと直列化を併用したい場合に便利である。

### バリア

``` python
threading.Barrier(parties, action=None, timeout=None)
```

これはバリアオブジェクトを返す。**バリア**は、指定された数のスレッドが到達するまで待つような地点を設定する機能を提供する。

具体的には、コンストラクタに渡される `parties` がスレッドの必要数であり、バリアオブジェクトの `wait()` メソッドでバリアの地点が設定される。つまり、`wait()` を呼んだ地点でスレッドが待機し、`parties` 個のスレッドが `wait()` を呼ぶと、それらは同時にすべて解放される。呼び出し可能オブジェクトが `action` としてコンストラクタに渡されていれば、スレッドが解放される時にそのうちの 1 つによって呼ばれる。

実のところ、バリアは内部にコンディションを持ち、スレッドの待ち合わせと解放がコンディションを使ったスレッド間通信により実現されている。このため、必要な数のスレッドがバリアの地点に到達しないと、デッドロックが発生する。デッドロックを回避するには、タイムアウトを使用する必要がある。`wait()` は `timeout` 引数を取り、それに浮動小数点数を与えると `timeout` 秒後にタイムアウトし、バリアが破壊される。スレッドが待っている間にバリアが破壊された場合、`wait()` メソッドは `threading.BrokenBarrierError` 例外を送出する。コンストラクタの `timeout` 引数でデフォルトのタイムアウト時間を指定できる。

次のコードはバリアの使用例である。

In [ ]:
import random
import threading
import time


def report():
    print("{}: passed through the barrier".format(threading.current_thread().name))


def worker(barrier):
    time.sleep(random.uniform(0.2, 1.0))
    print(f"{threading.current_thread().name}: start")
    try:
        barrier.wait(timeout=1.0)
    except threading.BrokenBarrierError:
        print(f"{threading.current_thread().name}: TIMEOUT")
    else:
        print(f"{threading.current_thread().name}: end")


def main():
    barrier = threading.Barrier(2, action=report)
    threads = [threading.Thread(target=worker, name=f"t{i}", args=(barrier,)) for i in range(5)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


if __name__ == "__main__":
    main()

t4: start
t2: start
t2: passed through the barrier
t2: end
t4: end
t1: start
t3: start
t3: passed through the barrier
t3: end
t1: end
t0: start
t0: TIMEOUT


### スレッドローカルデータ

`threading.local` は、**スレッドローカルデータ**と呼ばれ、スレッド間で共有するオブジェクトでありながら、属性値は共有されないという特殊なオブジェクトの型である。`threading.local` の使用では、競合状態が起こらない。

スレッドローカルデータの例としては、各スレッドの動作を記録するログが挙げられる。ログには次のようなデータが記録される。

  * スレッド名
  * セッション ID
  * リクエスト ID
  * エラー情報

これらはスレッドに固有であり、スレッドの存続期間に渡って使用されるデータであるが、スレッドの実行に必要ではないため、スレッドで実行する関数の引数として渡したり、戻り値や例外メッセージとして呼び出し元に返すのは面倒である。この場合の一般的な解決策は、スレッドローカルデータを使用することである。

次のコードでは、`threading.local` を使用する例である。`local` は `threading.local` のインスタンスで、`worke` スレッドで `x` 属性が変更されるにもかかわらず、メインスレッドでは `x` 属性の変更が共有されない。

In [ ]:
import threading

local = threading.local()


def worker():
    local.x = 1
    print(f"{threading.current_thread().name}: {local.x=}")


def main():
    local.x = 0

    thread = threading.Thread(target=worker, name="worker")
    thread.start()
    thread.join()

    print(f"{threading.current_thread().name}: {local.x=}")


if __name__ == "__main__":
    main()

worker: local.x=1
MainThread: local.x=0


### スレッドのカスタマイズ

`threading.Thread` クラスを継承して、`run()` メソッドをオーバーライドしたサブクラスを利用することができる。もとの `run()` メソッドは、インスタンス化の際に `target` が渡されていれば、それを実行するだけである。`run()` メソッドをオーバーライドすることで、スレッドの動作をカスタマイズできる。

クラス継承を使用したカスタマイズの良い例が、`threading.Timer` である。`threading.Timer` は、`interval` 秒後に実行するスレッドである。次のコードは、その定義の要約である。

``` python
class Timer(Thread):
    def __init__(self, interval, function, args=None, kwargs=None):
        Thread.__init__(self)
        self.interval = interval
        self.function = function
        self.args = args if args is not None else []
        self.kwargs = kwargs if kwargs is not None else {}
        self.finished = Event()

    def cancel(self):
        """Stop the timer if it hasn't finished yet."""
        self.finished.set()

    def run(self):
        self.finished.wait(self.interval)
        if not self.finished.is_set():
            self.function(*self.args, **self.kwargs)
        self.finished.set()
```

`threading.Timer` は、内部にイベントオブジェクト `finished` を持ち、`run()` メソッドが `wait()` でタイムアウトすると `function` を実行するようにオーバーライドされている。`interval` 秒経過する前であれば中止できる `cancel()` メソッドも定義されている。

たとえば、次のコードを実行すると、30 秒後に `'hello, world'` が出力される。

``` python
def hello():
    print("hello, world")

t = Timer(30.0, hello)
t.start()
```

queue
-----

標準ライブラリの `queue` モジュールは、データ構造とコンディションを組み合わせて、スレッド間で共有するデータ構造から次々とアイテムを取り出し、アイテムに対する処理をロックを伴って行うことをサポートする。これは、複数のスレッドの間でデータを安全に交換しなければならないときのマルチスレッドプログラミングで特に有益である。

内部に持つデータ構造の種類により、以下のクラスが提供される。

  * `queue.Queue`: FIFO のデータ構造、すなわちキュー（`collections.deque` で実装される）
  * `queue.LifoQueue`: LIFO のデータ構造、すなわちスタック（`collections.deque` で実装される）
  * `queue.PriorityQueue`: 優先度付きキュー（`heapq` の関数で実装される）

下 2 つのクラスは `queue.Queue` の派生クラスであり、データ構造に関する内部メソッドをオーバライドしている。3 つのクラスのコンストラクタ引数とメソッドは共通している。

``` python
queue.Queue(maxsize=0)
```

コンストラクタの引数 `maxsize` は、キューに入れられるアイテム数の上限を設定する整数。`maxsize` が 0 以下の場合は、キューの大きさは無限となる。

`queue.Queue` は、キューに入れられたアイテムで処理が完了していないものを内部でカウントする。このカウントを「未完了タスクのカウント」と呼ぶ。アイテムをキューに入れるとき、「未完了タスクのカウント」はインクリメントされる。アイテムの処理が完了したら、「未完了タスクのカウント」をデクリメントできる。

主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `put(item, block=True, timeout=None)` | `item` をキューに挿入し、「未完了タスクのカウント」をインクリメントする。キューの上限が設定されていて<br /><br />上限に達していた場合、`block` が `True`（デフォルト）なら挿入処理はキューのアイテムが取り出されて空<br /><br />きが出るまでブロックされる。ただし、`timeout` に浮動小数点数を指定していれば、空きが出ないまま<br /><br /> `timeout` 秒経過した時に `queue.Full` 例外を送出する。`block` が `False` なら直ちに `queue.Full` 例<br /><br />外を送出する（この場合 `timeout` は無視される） | `None` |
| `put_nowait(item)` | `put(item, block=False)` と等価 | `None` |
| `get(block=True, timeout=None)` | キューからロックを伴ってアイテムを取り出す。通知を受けるか、タイムアウトするまで待機する。キューが空<br /><br />の場合は、`block` が `True`（デフォルト）ならアイテムが入るまでブロックされ、`block` が `False` なら直ち<br /><br />に `queue.Empty` 例外を送出する（この場合 `timeout` は無視される） | アイテム |
| `get_nowait()` | `get(block=False)` と等価 | アイテム |
| `empty()` | キューが空の場合は `True` を返し、そうでなければ `False` を返す | `bool` |
| `full()` | キューが一杯の場合は `True` を返し、そうでなければ `False` を返す | `bool` |
| `task_done()` | 内部コンディションの `notify_all()` を呼び出し、`get()` で待機中の全てのスレッドを起こす。また、「未完<br /><br />了タスクのカウント」をデクリメントする。この結果 0 未満になった場合（つまりキューにある要素より多く呼<br /><br />び出された場合） `ValueError` 例外が発生する | `None` |
| `join()` | 「未完了タスクのカウント」が 0 になるまで待機する | `None` |

`join()` はキューにあるすべてのアイテムが取り出されて処理されるまで待機したい場合に使用する。この場合、アイテムの取り出しに使われた `get()` の後に必ず `task_done()` を呼び出すようにしないと、いつまでたっても「未完了タスクのカウント」が 0 にならず、`join()` で待機し続けることになるので注意。

次のコードは、5 つのアイテム（`1` から `5` までの整数）の処理を 2 つのスレッドで同時に行うためにキューを使用する例である。各スレッドが動かす `worker()` 関数は、while ループにより、キューからアイテムを取り出し処理が終わっても終了せず、キューが空になるまでアイテムの処理を繰り返す。このように、ある一定数のスレッドをあらかじめ作成し、必要に応じてタスクを割り振ったり、使い回したりする仕組みを**スレッドプール**（thread pool）と呼ぶ。

In [ ]:
import queue
import threading
import time


def worker(q):
    while not q.empty():
        try:
            item = q.get_nowait()
        except queue.Empty:
            break
        else:
            print(f"{threading.current_thread().name}: Working on {item}")
            time.sleep(item * 0.1)
            print(f"{threading.current_thread().name}: Finished {item}")
            q.task_done()


def main():
    # Queue を設定
    q = queue.Queue()
    for i in range(1, 6):
        q.put(i)

    # スレッドプールを設定（2 つのスレッドからなる）
    threads = []
    for i in range(2):
        thread = threading.Thread(target=worker, name=f"t{i}", args=(q,))
        threads.append(thread)

    # スレッドの開始
    for thread in threads:
        thread.start()

    # キューが空になるまで待機
    q.join()
    print("全てのタスクが完了した")

    # 終了を待機
    for thread in threads:
        thread.join()


if __name__ == "__main__":
    main()

t0: Working on 1
t1: Working on 2
t0: Finished 1
t0: Working on 3
t1: Finished 2
t1: Working on 4
t0: Finished 3
t0: Working on 5
t1: Finished 4
t0: Finished 5
全てのタスクが完了した


`worker()` 関数では while の条件でキューが空でないことを確認しているが、`get_nowait()` でキューが空の場合の例外処理をしている。確認後にキューの状態が他のスレッドにより変更される可能性があるからである。`get_nowait()` はキューが空の場合に直ちに `queue.Empty` 例外を送出する。`queue.Empty` 例外を捕捉したら while ループを停止して関数を終了する（else 節は実行されない）。

実行結果から、各タスクが処理の完了したスレッドに次々と割り振られ、各スレッドではタスク処理が直列化されることがわかる。

スレッドセーフ
--------------

マルチスレッドでデータ競合による未定義動作を引き起こしたり、競合状態による意図しない動作を引き起こしたりせず、デッドロックなどの問題も起きないプログラムは**スレッドセーフ**（thread-safe）であるという。

以下は、Python でのマルチスレッドとスレッドセーフなコーディングのためのベストプラクティスである。

  * **マルチスレッドを使わない**  
マルチスレッドはプロセス内の制御フローが難解になる。メモリ共有のため、1 つのスレッドの問題がプロセス全体に影響を与える可能性がある。同期やロックを忘れたり、デッドロックが起こるというミスを防げない。再現性の低いバグを見つける作業は難しい。CPython 特有の問題として、CPU バウンドな処理ではかえって遅くなる。マルチプロセスを使うべきである。
  * **共有リソースを最小化する**  
あえてマルチスレッドを選択する場合（マルチプロセスでのオーバーヘッドが許容できないなど）、共有データを使用せず、同期やロックを考えなくて済むようにすること。スレッドローカルデータを活用すること。
  * **不変オブジェクトを利用する**  
値オブジェクトは上記の例外である。値オブジェクトは不変だから、自然とスレッドセーフである。
  * **ロックの範囲を最小化する**  
どうしても共有データを使わなければならない場合、ロックを獲得している間は、できるだけ少ない操作を行い、他のスレッドが待たされる時間を短縮すること。
  * **タイムアウトを使用する**  
`acquire()` と `wait()` にタイムアウトを設定することで、デッドロックが発生した場合でも、スレッドが適切に解放されるようになる。